In [1]:
import numpy as np
import math






# Key generation

In [2]:
class keygenertion:
    
    "Geneartion of private and public key for message of size noOfBits that will be sent"
    def __init__(self, noOfBits):
        self.nb = noOfBits
        "generation of n and k" 
        self.n = 2**self.nb -1
        self.k = 2**self.nb - self.nb -1
        
        "(n,k)-linear Hamming code with a fast decoding algorithm that can correct t errors. A generator matrix for this code is given by self.G"
        
        "Generation of Hamming codes"
        self.G = self.genMatrix()
        "Generates an invertable matrix of size k * k"
        self.S = self.genSMatrix() #
        
        "Generates a permutation matrix of size n * n using the given sequence"
        self.P = self.genPMatrix()
        
        "Computes Creates the GPrime encrytion k*n matrix "
        self.GPrime = self.genGPrimeMatrix()
        "public key is GPrime"
        "private key is  (S,G,P)"
        self.printCode()
        
    def genMatrix(self):
        "Generate Hamming Code Parity-Check and Generator Matrices"
        "For more details about the algorithm, see Key generation section in https://en.wikipedia.org/wiki/McEliece_cryptosystem"
        left = np.zeros((self.nb, 2**self.nb - 1 - self.nb)).T
        row = 0
        for j in range(2**self.nb):
            if j + 1 != 1:
                if (j + 1) & j != 0:
                    string_repres = np.binary_repr(j+1)
                    column = np.zeros((len(string_repres), 1))
                    for j in range(len(string_repres)):
                        column[-j - 1] = string_repres[j]
                    "perform padding in column according to pad_width= (0, self.nb - len(string_repres))"    
                    column = np.pad(column, (0, self.nb - len(string_repres)), 'constant')
                    left[row] = column.T[0]
                    row += 1
        left = left.T
        
        "parity-check matrix block"
        self.block = np.block([left, np.identity(self.nb)])
        
        "return the code generator matrix self.G"
        return np.block([np.identity(self.k), np.transpose(left)])

    def genSMatrix(self):
        "Generates an invertible matrix S"
        S = np.random.randint(0,2,(self.k, self.k), int)
        "np.matrix(np.random.randint(0,2,self.k*self.k).reshape(self.k,self.k).astype(int))"
        while np.linalg.det(S) == 0:
            S = np.random.randint(0,2,(self.k, self.k), int)
        return S
    
    

    def genPMatrix(self):
        "Geneartes a random permutation of the identity matrix, P"
        "P is an n × n matrix which has a single 1 in each row and column and 0's everywhere else."
        P = np.identity(self.n, int)
        P=P[np.random.permutation(self.n)]
        return P

    def  genGPrimeMatrix(self):
        "Creates the GPrime encrytion Matrix"
        return np.matmul(np.matmul(self.S, self.G), self.P) % 2
    
    def printCode(self):
        "Print codes to screen" 
        print ("S: \n" + str(self.S) + "\n")
        print ("P: \n" + str(self.P) + "\n")
        print ("GPrime: \n" + str(self.GPrime) + "\n")

# Encryption

In [3]:
class Encryption:
    "When given a message msg will be encrypted by using the public key GPrime"    
    "Errors will be added to the message" 
    "the message msg will be encoded as a binary string of length k"
    def __init__(self, msg, GPrime, t=1):
        self.GPrime = GPrime
        self.msg = msg
        (k, n) = GPrime.shape
        self.n = n
        self.k = k
        self.t = t
        self.z = self.genErrors()
        self.encryp_msg = self.encode()

    def genErrors(self):
        "Generates a random n-bit vector z (binary string) containing exactly t ones (a vector of length n and weight t)"
        zz = np.zeros(self.n)
        liste = np.random.choice(self.n, self.t, replace=False)
        for i in liste:
            zz[i] = 1
        return zz

    def encode(self):
        "computes the ciphertext by multiplying by GPrime, and add random error"
        "For more details about the algorithm, see message encryption section in https://en.wikipedia.org/wiki/McEliece_cryptosystem"
        self.CPrime = np.matmul(self.msg, self.GPrime) % 2 
        c = (self.CPrime + self.z) % 2 
        return c # ciphertext

    def getOriginalMessage(self):
        return self.msg

    def getEncryptedMessage(self):
        return self.encryp_msg


# Decryption

In [4]:
class Decryption:
    "When given ciphertext will decode to message"
    
    def __init__(self, ciphertext, S, P, block, msg):
        self.ciphertext = ciphertext
        self.S = S
        self.P = P
        self.block = block
        self.msg = msg
        self.decryp_msg = self.decrypt()
        
    def decrypt(self):
        "Decrypt a given message msg using the private key (S,G,P)"
        
        "Computes the inverse of P"
        P_inverse = np.linalg.inv(self.P)
        
        "Computes the inverse of S"
        S_inverse = np.linalg.inv(self.S)
        
        "Computes Cprime, for more information, see the following link, https://en.wikipedia.org/wiki/McEliece_cryptosystem"
        CPrime = np.matmul(self.ciphertext, P_inverse)
        
        "Computes msg Prime"
        msgPrime = self.correctError(CPrime)
        
        "Computes decyrpted_msg"
        decyrpted_msg= np.matmul(msgPrime, S_inverse) % 2
        
        "For more details about the algorithm, see message decryption section in https://en.wikipedia.org/wiki/McEliece_cryptosystem"
       
        
        return decyrpted_msg


    def correctError(self, CPrime):
        
        "Calculation of parity matrix to find the error"
        parity_matrix = np.matmul(CPrime, np.transpose(self.block)) % 2
        
        parity_bits = np.ma.size(parity_matrix, 0)
        syndrome = 0
        
        "Calculation of the Syndrome (From bitstring to integer)"
        
        "The receiver multiplies parity_check matrix and the received codeword to obtain the syndrome vector, which will" 
        "indicate whether an error has occurred, and if so, for which codeword bit. Performing this multiplication again"
        
        for j in range(parity_bits):
            syndrome += 2**j * parity_matrix[j]
        
        
        "In the case of no error is found, return the message"
        if (int((syndrome - 1)) & int(syndrome)) == 0:
            return CPrime[0:(CPrime.size - parity_bits)]
        
        
        else:
            "In the case of error is detected, an error correction process will be performed"
            err_msg = CPrime
            err_bit = int(syndrome - math.ceil(np.log2(syndrome)) - 1)
            if err_msg[err_bit] == 1:
                err_msg[err_bit] = 0
                return err_msg[0:(CPrime.size - parity_bits)]
            elif err_msg[err_bit] == 0:
                 err_msg[err_bit] = 1
                 return err_msg[0:(CPrime.size - parity_bits)]
           

    def decyrptionSuccess(self):
        "Test if encryption and decyrption process have been done successuly"
        
        success = (self.msg == self.decryp_msg)
        if success.all :
            print("\n")
            print("------------- Result ---------------")
            print ("Decryption has been done successufly !!")
            
            
    def getDecryptedMessage(self):
        return self.decryp_msg

## McEliece Crypto System

In [5]:
import numpy as np

print ("------------- Key Generation ---------------")
key = keygenertion(3)

"Get the public key (GPrime) that will be used to encrypt the message"
GPrime = key.GPrime

"Generate input message that will be encrypted"
message=np.array([1,0,1,0])

print ("------------- ENCRYPTION ---------------")

"Hamming encode messages of specified length 4"
encrypt = Encryption(message, GPrime)
ciphertext= encrypt.encode()
print("Original message = " + str(message))
print("Error Introduced =" + str(encrypt.z))
print("Encrypted Message = " + str(encrypt.getEncryptedMessage()))

print ("------------- DECRYPTION ---------------")


"Hamming encoded messages of specified length"
decrypt = Decryption(ciphertext, key.S, key.P, key.block, message)
print("Decrypted message = " + str(decrypt.getDecryptedMessage()))
decrypt.decyrptionSuccess()



------------- Key Generation ---------------
S: 
[[0 1 0 0]
 [1 0 0 0]
 [0 1 1 0]
 [1 0 1 1]]

P: 
[[0 0 0 0 1 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 0 0 1 0]
 [1 0 0 0 0 0 0]
 [0 1 0 0 0 0 0]
 [0 0 0 0 0 0 1]
 [0 0 1 0 0 0 0]]

GPrime: 
[[0. 1. 1. 1. 0. 0. 0.]
 [0. 1. 0. 0. 1. 0. 1.]
 [0. 1. 0. 1. 0. 1. 1.]
 [1. 0. 0. 0. 1. 1. 1.]]

------------- ENCRYPTION ---------------
Original message = [1 0 1 0]
Error Introduced =[0. 0. 0. 1. 0. 0. 0.]
Encrypted Message = [0. 0. 1. 1. 0. 1. 1.]
------------- DECRYPTION ---------------
Decrypted message = [1. 0. 1. 0.]


------------- Result ---------------
Decryption has been done successufly !!
